# Workshop: Introduksjon til dataflyt og transformasjon

**Du vil lære:**
- Helt overordnet hva dataflyt og transformasjon er, hva det innebærer og hvordan det utføres i praksis
- Litt om Google Cloud Storage og Google BigQuery
- Å laste inn, hente ut og jobbe med data fra Google BigQuery
- Gjøre enkle transformasjoner ved hjelp av datamanipuleringsverktøy
- Lage et nytt og rikere datasett med data fra flere kilder 

**Du vil _ikke_ lære:**
- Hvordan man setter opp en faktisk dataflyt eller lignende i Google Cloud Platform
- Spesifikke detaljer om verktøy som Google Cloud Platform (GCP), Pandas, Matplotlib, Keras, Tensorflow og liknende


💡 Oppgave 1-4 er utforskende i GCP og kan timeboxes til ca. 10 minutter.


# Relevante GCP-komponenter for workshopen
## Google Cloud Storage

Google Cloud Storage (GCS) er rett og slett et filområde hvor vi kan lagre filer på ulike formater, med både strukturerte og ustrukturerte data. GCS er et fint landingspunkt for data, slik at du kan jobbe uavhengig av systemene dataene stammer fra (f.eks eksterne APIer). 


Vi bruker ofte begrepet "storage bucket" for en logisk oppdeling av et filområde (noe tilsvarende som en filmappe på din maskin).

Data i en storage bucket er som regel ikke klargjort for analyseformål. Vi ønsker derfor å prosessere og flytte den til et annet verktøy. Et slikt verktøy kan være Google BigQuery, som er en database tilpasset analyse. Dette kan videre kobles opp mot visualiserings- og modelleringsverktøy som Colab-notebooks.

## Google BigQuery

BigQuery er en SQL-basert database som er optimalisert for analyse. I motsetning til tradisjonelle SQL-databaser er BigQuery kolonnebasert istedenfor radbasert. Dette gjør den optimalisert til å regne ut aggregerte tall. BigQuery takler tabeller med et høyt antall kolonner svært godt.


# Oppgaver




## Oppgave 1: Bli kjent med Google Cloud Storage (GCS)

Gå til [Google Cloud Console](https://console.cloud.google.com), logg inn med din epost og velg prosjekt "data-intro" oppe i venstre hjørne.

I Google Cloud Console (GUI) for prosjektet (data-intro), finn Google Cloud Storage. Du kan enten finne GCS via menyen eller søke etter den i søkefeltet. 

I prosjektet finner du en bucket med to ulike datasett. Hva finner du ut om disse datasettene (metadata)?

a) Hvilken filtype er de?
  
  
b) Hvor store er filene?
  

In [ ]:
# Svar a:
# Svar b:

## Oppgave 2: Importer dataen til BigQuery

For å se nærmere på innholdet i datasettene ønsker vi å flytte de til BigQuery. Gjør følgende i [Google Cloud Console](https://console.cloud.google.com):

1. Finn BigQuery i menyen
2. Velg BigQuery-prosjektet "Data intro" og deretter marker datasettet bysykkel_main
3. I menylinjen oppe til høyre, velg "Create table". 
4. Under "Source" kan du velge datakilden din. Vi ønsker å velge bysykkeldatasettet fra Storage Bucket. Filformatet fant du i oppgave 1. 
5. Under "Destination" kan du kalle den nye tabellen din `bysykkel_(gruppenavn)`. 
6. La BigQuery definere skjema for deg, og behold ellers standard innstillingene.
7. Trykk på "Create Table" 

Datasettet er nå lastet inn i BigQuery.

## Oppgave 3: Bli kjent med datasettet

Når vi markerer tabellen i BigQuery som vi laget i forrige oppgave ser vi en rekke metadata, samt en preview-funksjon for å undersøke radene i datasettet vårt.

Hva finner du ut om skjema (datatypene) og innholdet?




## Oppgave 4: Værdata!
Vi har allerede lastet inn værdatasettet inn i BigQuery (`værdata_oslo`).
Hva finner du ut om skjema (datatypene) og innholdet for dette datasettet?



## Oppgave 5: Lage et utvidet datasett
Vi ønsker å slå sammen de to datasettene slik at vi kan gjøre analyse på tvers av disse. 

💡 **NB! Du må jobbe i din egen notebook-kopi om du ikke allerede har gjort det. Se avsnittet "Hvordan komme i gang" på Github for detaljer**

1. Koble deg til BigQuery. Først må du autentisere deg (med din Bekk Google bruker som har tilgang til BigQuery). Kjør kodesnutten under for å gjøre dette.


In [ ]:
# Authenticate your Google Account
# Doing so means you have access to various
# resources connected to your account, such
# as BigQuery tables, Storage buckets etc.
from google.colab import auth
auth.authenticate_user()

Under har vi en hjelpemetode for å laste inn datasettene dine fra BigQuery. Kjør denne kodesnutten også. 

In [ ]:
# Eksterne avhengigheter
from google.cloud import bigquery_storage
from google.cloud.bigquery_storage import types
from google.cloud.bigquery_storage_v1 import enums
import pandas

#  params:
#   project_id: String
#   dataset_id: String
#     table_id: String
# 
#  return:
#           df: Pandas DataFrame
#
def load_bigquery_data(project_id, dataset_id, table_id):
    # Parse input-verdier til forventet filsti på Google BigQuery
    table = f"projects/{project_id}/datasets/{dataset_id}/tables/{table_id}"
    parent = "projects/{}".format(project_id)

    # Instansier klient for enkel integrasjon mot BigQuery
    bqstorageclient = bigquery_storage.BigQueryReadClient()

    # Opprett en read-session mot en tabell i BigQuery
    requested_session = types.ReadSession(
        table=table,
        data_format = enums.DataFormat.ARROW
    )
    read_session = bqstorageclient.create_read_session(
        parent=parent,
        read_session=requested_session,
        max_stream_count=1,
    )

    # Les data fra BigQuery, putt i en liste med dataframes
    stream = read_session.streams[0]
    readRowsStream = bqstorageclient.read_rows(stream.name)
    dfs = []
    for page in readRowsStream.rows(read_session).pages:
        dfs.append(page.to_dataframe())

    # Sett flere dataframes sammen til én
    df = pandas.concat(dfs)

    return df

2.  Når du er autentisert, laster du inn bysykkel-datasettet til gruppen din, samt værdatasettet inn i notebooken. Datasettene blir lastet inn som dataframes (som du kan lese mer om [her](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)). 

Vi kaller dataframesene henholdsvis `df_bysykkel` og `df_weather`.

Kjør kodelinjene under og undersøk innholdet.

In [ ]:
## Husk å bytte ut med gruppen din sitt bysykkeldatasett
df_bysykkel = load_bigquery_data("data-intro", "bysykkel_main", "bysykkel_[DITT_GRUPPENAVN]")
df_bysykkel.head()


In [ ]:
df_weather = load_bigquery_data("data-intro", "bysykkel_main", "værdata_oslo")
df_weather.head()

3. Vi må finne en kolonne med fellesdata for å kunne slå sammen tabellene. Så du noen fellesnevnere da du undersøkte innholdet i tabellene?

In [ ]:
## DITT SVAR HER

4. Et alternativ er å slå sammen tabellene basert på dato. Værdatasettet har en gjennomsnittstemperatur og en gjennomsnittsnedbørsmengde for hver dato. På bysykkeldatasettet har vi flere kolonner som inneholder dato, så her må vi velge en. Valget avhenger av hva vi ønsker å analysere. Dette datasettet er ikke så omfattende, så i vårt eksempel velger vi å slå sammen på turens starttidspunkt (`started_at`).

  Vi forsøker å slå sammen tabellene ved å bruke `started_at` i bysykkeldatasettet og `date` i værdatasettet. [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)-funksjonen i pandas kan hjelpe oss.

  Kjør kodelinjen under. Støtte du på noen utfordringer? Hvorfor fungerer ikke dette?

In [ ]:

df_merged = pandas.merge(df_bysykkel, df_weather, left_on='started_at', right_on='date', how='left') 



<details><summary>🚨 Løsningsforslag</summary>

Ta en kikk på innholdet i disse to kolonnene. Ser det ut som datoformatet er det samme i de ulike tabellene? Her må vi gjøre mer transformasjon før vi kan fortsette!

</details>


## Oppgave 6: Rydde opp i datasettene
Transformasjoner er en stor og viktig prosess når vi jobber med data. Ofte er datasettene vi har til rådighet ikke på det formatet vi ønsker å ha de på. Å transformere data betyr å gjøre endringer, f.eks:
- slå sammen datasett
- endre på datatyper 
- fjerne duplikater 
- gjøre utregninger med basis i andre kolonner
- fjerne potensielle "outliers" som kan ødelegge grunnlaget vårt for analyse

> 🧼 Prosessen over kalles ofte for å "vaske" data.

Vi må få datokolonnene til å være på samme format. En måte vi kan gjøre dette på er å fjerne klokkeslettet og kun bruke datodelen av `started_at`. Ulempen med dette er at vi da mister informasjon vi kanskje ønsker å bruke videre i analyse/innsiktsdelen. 

Vi løser dette problemet med å lage en hjelpekolonne, altså en ny midlertidg kolonne som kun brukes når vi slår sammen datasettene

1. Lag en ny kolonne i `df_bysykkel`, `trip_date`, som kun inneholder datoen fra kolonnen `started_at`.

**💡 Tips**: Pandas har en funksjon [`to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) som lar deg tilpasse tidspunkter



In [ ]:
### DIN KODE HER
##
##

<details><summary>🚨 Løsningsforslag</summary>

```
df_bysykkel["trip_date"] = pandas.to_datetime(df_bysykkel["started_at"]).dt.strftime("%Y-%m-%d") 
df_bysykkel.head()

```

</details>

2. Forsøk å merge på nytt ved å bruke hjelpekolonnen `trip_date` ved å kjøre kodelinjen under. Fungerer det å slå sammen nå? Hvordan ser datasettet ut?

In [ ]:
df_merged = pandas.merge(df_bysykkel, df_weather, left_on='trip_date', right_on='date', how='left') 
df_merged.head()


<details>
<summary>🚨 Løsningsforslag</summary>  
Det vil i prinsippet fungere, men det er likevel ikke riktig fordi kolonnene har ulike datatyper. Du kan sjekke hvilke datatyper dataframen din inneholder ved å bruke pandas "dtypes", f.eks df_bysykkel.dtypes

Du kan lese mer om funksjonen dtypes [her](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html)

</details>



3. Gjør nødvendige endringer i kolonnen `trip_date` i bysykkeldatasettet og `date` i værdatasettet for å kunne slå sammen.


In [ ]:
## DIN KODE HER
##
##
##


<details><summary>🚨 Løsningsforslag</summary>
Vi setter begge kolonnen til å være av type datetime (datotid).

```
df_bysykkel["trip_date"] = pandas.to_datetime(df_bysykkel["trip_date"])
df_weather["date"] = pandas.to_datetime(df_weather["date"])

```

</details>

4. Nå skal det fungere å slå sammen datasettene! 🎉 

 Vi ønsker å supplere hver rad av bysykkeldatasettet med værdata fra den aktuelle dagen. Skriv kode under som oppnår dette. 
 Rekkefølgen og type join (left, right, outer etc) har noe å si når datasettene slås sammen.  




In [ ]:
## DIN KODE HER
##
##
##


<details>
<summary>🚨 Løsningsforslag</summary>  

```
df_merged = df_bysykkel.merge(df_weather, left_on='trip_date', right_on='date', how='left')
df_merged.head()

```

</details>


5. Kjør `dtypes` igjen. Værdata-kolonnene ser ikke ut til å være av typen tall, men string. Det vil ikke fungere å gjøre aggregeringer på strings. Hvorfor fungerer det ikke å gjøre om kolonnen direkte?

In [ ]:
df_merged["mean_temperature"] = df_merged["mean_temperature"].astype('float')
df_merged["precipitation_amount"] = df_merged["precipitation_amount"].astype('float')

<details>
<summary>🚨 Løsningsforslag</summary>

Kolonnen ser tilsynelatende ut til å kun bestå av tall. Hvis vi derimot inspiserer verdiene nærmere, ser vi at noen ganger forekommer strengen "NULL" som ikke er et tall. Pandas får derfor ikke til å gjøre om datatypen før vi gjør noe med dette.

</details>


6. Endre datatypen på nedbørskolonnen og temperaturkolonnen slik at det blir desimaltall.

In [ ]:
## DIN KODE HER
##
##

<details>
<summary>🚨 Løsningsforslag</summary>

Vi må rydde opp i disse kolonnene ved å fjerne strengene 'NULL' før vi kan endre datatypen til float. Vi kan bruke `replace` funksjonen til en dataframe.

Vi kunne gjort om NULL til 0, men dette ville fått betydning for statistiske verdier som gjennomsnitt. Vi velger derfor å sette verdien "blank", som heter "None" i Python.

```
df_merged["mean_temperature"] = df_merged["mean_temperature"].replace('NULL',None).astype('float')
df_merged["precipitation_amount"] = df_merged["precipitation_amount"].replace('NULL',None).astype('float')

```
</details>


7. Til slutt: Rydd opp ved å fjerne hjelpekolonnen fra `df_merged`


In [ ]:
## DIN KODE HER
##
##
##


<details>
<summary>🚨 Løsningsforslag</summary> 

```
df_merged = df_merged.drop(columns="trip_date")
df_merged.head()

```

</details>



Vi har nå et utvidet datasett! 🎉🥂🎊


Vanligvis ville vi ha skrevet datasettet tilbake til BigQuery, men det gjør vi ikke i denne workshopen. (Det tok lang tid, vi har mye data😴) 

Pandas har en ganske snedig funksjon som kan gjøre dette!

```df_merged.to_gbq("bysykkel_main.bysykkel_med_værdata", project_id="data-intro")```

Når vi skriver til BigQuery, vil datatypene i dataframen følge med og sette riktig skjema i BigQuery
